In [2]:
import numpy as np
import librosa
import sounddevice as sd
from tensorflow.keras.models import load_model

model = load_model("C:\\3-2\\p\\soundDetection\\car_detection_cnn_model2_48kHz.h5")


In [ ]:


# 오디오 전처리 함수
def preprocess_audio(file_path, max_pad_len=199, target_sample_rate=48000):
    """
    입력 오디오 파일을 모델 입력 형식에 맞게 전처리합니다.
    :param file_path: 입력 오디오 파일 경로
    :param max_pad_len: 고정된 입력 길이
    :param target_sample_rate: 오디오 샘플레이트
    :return: 전처리된 MFCC 특징 데이터
    """
    # 오디오 로드 및 리샘플링
    audio, sample_rate = librosa.load(file_path, sr=target_sample_rate, res_type='kaiser_fast')
    
    # MFCC 추출 (40개의 MFCC, 25ms 윈도우, 10.7ms 스텝)
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40, n_fft=2048, hop_length=512)
    
    # 패딩 또는 자르기
    pad_width = max_pad_len - mfcc.shape[1]
    if pad_width > 0:
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]
    
    # CNN 입력 형식으로 변환
    return mfcc[np.newaxis, ..., np.newaxis]

# 예측할 오디오 파일 경로
# test_audio_path = "C:\\3-2\\p\\soundDetection\\data\\splitCar\\car_temp_2_467.wav"
# test_audio_path = "C:\\3-2\\p\\soundDetection\\data\\splitCar\\car_temp_segment_100.wav"
# test_audio_path = "C:\\3-2\\p\\soundDetection\\data\\splitNoCar\\noCar_temp_2_45.wav"
# test_audio_path = "C:\\3-2\\p\\soundDetection\\data\\splitNoCar\\noCar_temp_segment_78.wav"
# test_audio_path = "C:\\Users\\jk070\\Downloads\\VehicleNoise1016.wav"
# test_audio_path = "C:\\Users\\jk070\\Downloads\\audio_1-34497-A-14.wav"

# 오디오 데이터 전처리
processed_audio = preprocess_audio(test_audio_path)

# 모델 예측
prediction = model.predict(processed_audio)

# 예측 결과 해석
prediction_value = prediction[0][0]
print(f"Prediction Value: {prediction_value:.8f}")
if prediction_value < 0.5:
    print("Prediction: Car sound detected!")
else:
    print("Prediction: No car sound detected!")


1/1 [==============================] - 0s 20ms/step
Prediction Value: 1.00000000
Prediction: No car sound detected!
